## Building A Chatbot
In this video We'll go over an example of how to design and implement an LLM-powered chatbot. This chatbot will be able to have a conversation and remember previous interactions.

Note that this chatbot that we build will only use the language model to have a conversation. There are several other related concepts that you may be looking for:

- Conversational RAG: Enable a chatbot experience over an external source of data
- Agents: Build a chatbot that can take actions

This video tutorial will cover the basics which will be helpful for those two more advanced topics.

In [1]:
from google.colab import userdata
GROQ_API_KEY = userdata.get('GROQ_API_KEY')

In [2]:

import os

In [3]:

groq_api_key=os.getenv("GROQ_API_KEY")

In [4]:
!pip install langchain_groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.6/109.6 kB 4.5 MB/s eta 0:00:00


In [5]:
from langchain_groq import ChatGroq
model=ChatGroq(model="Gemma2-9b-It",groq_api_key=GROQ_API_KEY)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7d7ee2c14950>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7d7ee2fa54d0>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [6]:
!pip install langchain_core

In [10]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi , My name is Cherin and I am a Chief AI Engineer")])

AIMessage(content="Hello Cherin, it's nice to meet you! That's a fascinating title. \n\nAs a Chief AI Engineer, I imagine you're deeply involved in some exciting projects.  \n\nWhat kind of work are you currently focusing on?  Is there anything in particular you'd like to discuss or learn about today?\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 71, 'prompt_tokens': 23, 'total_tokens': 94, 'completion_time': 0.129090909, 'prompt_time': 0.000146319, 'queue_time': 0.02044872, 'total_time': 0.129237228}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-09595e8c-ea5e-414c-8b93-fc421196d854-0', usage_metadata={'input_tokens': 23, 'output_tokens': 71, 'total_tokens': 94})

In [11]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi , My name is Cherin and I am a Chief AI Engineer"),
        AIMessage(content="Hello Cherin, it's nice to meet you! That's a fascinating title. \n\nAs a Chief AI Engineer, I imagine you're deeply involved in some exciting projects.  \n\nWhat kind of work are you currently focusing on?  Is there anything in particular you'd like to discuss or learn about today?\n"),
        HumanMessage(content="Hey What's my name and what do I do?")
    ]
)

AIMessage(content="You are Cherin, and you are a Chief AI Engineer.  \n\nIs there anything else you'd like to know about yourself or discuss? 😊 \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 35, 'prompt_tokens': 114, 'total_tokens': 149, 'completion_time': 0.063636364, 'prompt_time': 0.003619696, 'queue_time': 0.022432074, 'total_time': 0.06725606}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-5624eed8-a883-4aee-ab57-4d3a3cba9a62-0', usage_metadata={'input_tokens': 114, 'output_tokens': 35, 'total_tokens': 149})

In [ ]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi , My name is Cherin and I am a Chief AI Engineer")])

AIMessage(content="Hello Cherina, it's a pleasure to meet you!\n\nThat's a fascinating title. As a large language model, I'm always interested to learn more about the work that people in AI are doing. \n\nWhat kinds of projects are you working on as Chief AI Engineer?\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 63, 'prompt_tokens': 23, 'total_tokens': 86, 'completion_time': 0.114545455, 'prompt_time': 0.00015005, 'queue_time': 0.021614759, 'total_time': 0.114695505}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-0fac1bbf-9598-41b3-9bed-d1e4b2be6221-0', usage_metadata={'input_tokens': 23, 'output_tokens': 63, 'total_tokens': 86})

### Message History
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

In [12]:
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 30.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 63.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.2/412.2 kB 42.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 6.3 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.29
    Uninstalling langchain-core-0.3.29:
      Successfully uninstalled langchain-core-0.3.29
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.14
    Uninstalling langchain-0.3.14:
      Successfully uninstalled langchain-0.3.14


In [13]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model,get_session_history)

In [14]:
config={"configurable":{"session_id":"chat1"}}

In [15]:
response=with_message_history.invoke(
    [HumanMessage(content="Hi , My name is Cherin and I am a Chief AI Engineer")],
    config=config
)

In [16]:
response.content

"Hello Cherin, it's nice to meet you! That's a fascinating role. What kind of AI projects are you currently working on?  \n\nI'm eager to learn more about your work in the field.  Perhaps you could tell me about:\n\n* **The industry you're in:** Are you working in healthcare, finance, tech, or somewhere else?\n* **The types of AI models you develop:** Are you focused on machine learning, deep learning, natural language processing, or another area?\n* **Some of the challenges you face as a Chief AI Engineer:**\n\nI'm always learning and I'm sure I can learn a lot from your experience.\n"

In [17]:
with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

AIMessage(content='Your name is Cherin.  \n\nI remembered that from our first exchange! 😊  \n\nIs there anything else I can help you with?\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 32, 'prompt_tokens': 182, 'total_tokens': 214, 'completion_time': 0.058181818, 'prompt_time': 0.007047833, 'queue_time': 0.022079227, 'total_time': 0.065229651}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-c2895392-63b0-40c1-af5b-653f254a25fe-0', usage_metadata={'input_tokens': 182, 'output_tokens': 32, 'total_tokens': 214})

In [18]:
## change the config-->session id
config1={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

'As a large language model, I have no memory of past conversations and do not know your name. Please tell me your name so I can address you properly.\n'

In [19]:
response=with_message_history.invoke(
    [HumanMessage(content="Hey My name is John")],
    config=config1
)
response.content

"Hello John! It's nice to meet you. 😊 \n\nIs there anything I can help you with today?\n"

In [20]:
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

'Your name is John, as you told me earlier.  \n\nDid you forget?  😊  \n'

### Prompt templates
Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

In [21]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant.Answer all the question to the nest of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain=prompt|model

In [22]:
chain.invoke({"messages":[HumanMessage(content="Hi My name is Cherin")]})

AIMessage(content="Hi Cherin! It's nice to meet you.\n\nWhat can I do for you today? I'm ready to answer your questions and help in any way I can. 😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 43, 'prompt_tokens': 31, 'total_tokens': 74, 'completion_time': 0.078181818, 'prompt_time': 0.000326619, 'queue_time': 0.0216118, 'total_time': 0.078508437}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-686cdc3c-4d3c-4598-81a7-5966dc1563da-0', usage_metadata={'input_tokens': 31, 'output_tokens': 43, 'total_tokens': 74})

In [23]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [24]:
config = {"configurable": {"session_id": "chat3"}}
response=with_message_history.invoke(
    [HumanMessage(content="Hi My name is Cherin")],
    config=config
)

response

AIMessage(content="Hi Cherin, it's nice to meet you! \n\nWhat can I do for you today? I'm ready to answer your questions to the best of my ability. 😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 43, 'prompt_tokens': 31, 'total_tokens': 74, 'completion_time': 0.078181818, 'prompt_time': 0.000307489, 'queue_time': 0.019597780999999998, 'total_time': 0.078489307}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-fa588e1e-145c-47be-bbf1-3d576ad988d5-0', usage_metadata={'input_tokens': 31, 'output_tokens': 43, 'total_tokens': 74})

In [25]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your name is Cherin, as you told me!  😄 \n\n\n'

In [26]:
## Add more complexity

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [28]:
response=chain.invoke({"messages":[HumanMessage(content="Hi My name is Cherin")],"language":"French"})
response.content

"Bonjour Cherin, \n\nEnchanté de faire votre connaissance ! Comment puis-je vous aider aujourd'hui ? 😊 \n\n"

Let's now wrap this more complicated chain in a Message History class. This time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history.

In [29]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [31]:
config = {"configurable": {"session_id": "chat4"}}
repsonse=with_message_history.invoke(
    {'messages': [HumanMessage(content="Hi,I am Cherin")],"language":"Spanish"},
    config=config
)
repsonse.content

'¡Hola Cherin! 👋  \n\nMucho gusto. ¿Cómo puedo ayudarte hoy? 😄 \n'

In [32]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "French"},
    config=config,
)

In [33]:
response.content

'Votre nom est Cherin. 😊 \n'

### Managing the Conversation History
One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.
'trim_messages' helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages

In [34]:
from langchain_core.messages import SystemMessage,trim_messages
trimmer=trim_messages(
    max_tokens=45,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)
messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
trimmer.invoke(messages)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [41]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    | prompt
    | model

)


In [42]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response.content

'You asked "what is 2 + 2". 😊  \n\n\n\nIs there anything else I can help you with?\n'

In [44]:
response = chain.invoke(
    {
    "messages":messages + [HumanMessage(content="What ice cream do i like")],
    "language":"English"
    }
)
response.content

"As a helpful assistant, I don't have access to your personal information, including your taste preferences. \n\nWhat's your favorite flavor of ice cream?  🍦 😊\n"

In [45]:
## Lets wrap this in the MEssage History
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)
config={"configurable":{"session_id":"chat5"}}

In [46]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

"As a helpful assistant, I don't have access to any personal information about you, including your name.  \n\nIf you'd like to tell me your name, I'd be happy to use it! 😊\n"

In [39]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what math problem did i ask?")],
        "language": "English",
    },
    config=config,
)

response.content

"As a large language model, I have no memory of past conversations. If you'd like to ask me a math problem, I'm happy to help! 😊  \n\nWhat's your problem?  \n"